In [ ]:
%matplotlib widget

import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from python.fluorophores import FlStatic
from python.fluorophores import FlMoving
from python.fluorophores import FlCollectionBlinking
from python.psfs import PsfVectorial
from python.estimators import est_quad2Diter
from python.simulators import Simulator
from python.simulators import SimSequencefile
from python.tools import makeNPC

## Imaging of DNA-PAINT (blinking fluorophores + diffusive background) with Abberior scouting sequence

In [ ]:
sim = SimSequencefile()
sim.posgalvo = [0, 0, 0]
sim.posEOD = [0, 0, 0]
sim.time = 0
fname = os.path.join(SCRIPT_DIR, "examples", "Imaging_2D.json")
fname2 = os.path.join(SCRIPT_DIR, "settings", "PSFvectorial2D.json") # use a PSF that is defined via a json file
sim.loadsequence(fname, fname2)

sim.makescoutingpattern(np.array([[-80, -150], [120, 100]]))  # for imaging
sim.sequence['locLimit'] = 100  # to avoid getting stuck with background fluorophore

In [ ]:
maxtime=60*1e3 # ms, i.e. 1 min
cfrcutoff = 0.5

# make a fluorophore collection with blinking fluorophores
fc = FlCollectionBlinking()


laserpower = 8
switchpar = {}
switchpar['brightness'] = 100*laserpower
switchpar['toffsmlm'] = 20*1e3  # on-switching time in ms
switchpar['photonbudget'] = 8000
switchpar['tonsmlm'] = 100  # ms 
switchpar['activations'] = 1e18  # re activations (should be infinite, but this is too large for np.random.poisson())
switchpar['starton'] =-1  # fluorophores start in random on / off state, determined by tonsmlm, toffsmlm
fc.setpar(**switchpar)

# add fake NPCs
fc.add(makeNPC(pos=[0, 0, 0]))

# make diffusing molecules, back of the envelope calculations
# standard sequences: ~1 nM, fast sequences: 0.2 pM
# 1 M = Na/liter, 1 l= (0.1 m)^3 = (0.1 *1e6 um)^3 =1e15 um^3
# density of fluorophores (um^-3) for 1nM: 1e-9* 6e23/1e15=0.6 %density of fluorophores, about 1 / um^3
# slow sequences, bounding box: 1 um  x 1 um x 2 um: ~ 1 particle
# fast sequences: 2 um x 2 um x 2 um: ~1 particle
boundingbox = [2000, 2000, 2000]  # two fluorophores per 2x2x2 um
D=30  # um^2/s
fd=FlMoving()
fd.makediffusion(D,0.01,dim=3,boundarybox=boundingbox)
fd2=FlMoving()
fd2.makediffusion(D,0.01,dim=3,boundarybox=boundingbox)
fc.add([fd,fd2])

sim.fluorophores=fc

In [ ]:
brightnesses = [0, 1]  # compare without and with background from diffusing fluorophore
titles=["DNA-PAINT: imaging strands invisible", "DNA-PAINT: diffusive imaging strands"]

fig = plt.figure()

for k in range(len(brightnesses)):
    sim.posgalvo = [0, 0, 0]
    sim.posEOD = [0, 0, 0]
    sim.time = 0
    fc.reset()  # switch on all fluorophores again
    fd.brightness=switchpar['brightness']*brightnesses[k]
    fd2.brightness=switchpar['brightness']*brightnesses[k]
    
    out = sim.scoutingSequence(maxtime=maxtime)
    
    # plot results
    vld = (out.loc.vld==1) & (out.loc.itr==max(out.loc.itr))
    vldcfr = vld & (out.loc.cfr<cfrcutoff)
    notvld = (~vld) & (~vldcfr)

    # subplot(2,2,k); 
    ax = fig.add_subplot(221+k)
    ax.plot(sim.scoutingcoordinates[:,0],sim.scoutingcoordinates[:,1],'k*')
    ax.plot(out.loc.xnm[notvld],out.loc.ynm[notvld],'y.')
    ax.plot(out.loc.xnm[vld],out.loc.ynm[vld],'m.')
    ax.plot(out.loc.xnm[vldcfr],out.loc.ynm[vldcfr],'bx')
    posfl = out.fluorophores.pos[-1,:-2,:].squeeze()  # last one is diffusing
    ax.plot(posfl[:,0],posfl[:,1],'ro')
    ax.set_aspect('equal')
    ax.legend(['scouting','not vld', 'last itr vld','last itr vld +cfr', 'fluorophore'])
    ax.set_title(titles[k])
    ax.plot([-90, 10],[-70, -70],'k')
    ax.text(-50,-60,"100 µm")
    ax.set_axis_off()
    ax.set_xlim([-95, 100])
    ax.set_ylim([-85, 105])

## Imaging of blinking fluorophores with Abberior sequence

In [ ]:
# make abberior simulator

photonbudget = [800, 5000]
reactivations = [0, 2]
brightnesses = [50, 100]
titles = ["PALM", "dSTORM"]

offset = len(brightnesses)
for k in range(len(photonbudget)):
    sim.posgalvo = [0, 0, 0]
    sim.posEOD = [0, 0, 0]
    sim.time = 0
    fc = FlCollectionBlinking()
    # set parameterst for caged fluorophore, PAFP or similar
    switchpar = {}  # TODO: This cancels toff values, keep?
    switchpar['brightness']=brightnesses[k]*laserpower
    switchpar['toffsmlm'] = 20*1e3  # on-switching time in ms
    switchpar['photonbudget'] = photonbudget[k]
    switchpar['tonsmlm'] = 1e8 #  ms stays on, only bleached
    switchpar['activations'] = reactivations[k]  # re activations
    switchpar['starton'] = 0 # fluorophores start in random on / off state, determined by tonsmlm, toffsmlm
    fc.setpar(**switchpar)

    # add fake NPCs
    fc.addstatic(makeNPC(pos=[0, 0, 0]))
    fc.reset()

    sim.fluorophores = fc

    out = sim.scoutingSequence(maxtime=maxtime)

    # plot results
    vld = (out.loc.vld==1) & (out.loc.itr==max(out.loc.itr))
    vldcfr = vld & (out.loc.cfr<cfrcutoff)
    notvld = (~vld) & (~vldcfr)
    # subplot(2,2,k+2);
    ax = fig.add_subplot(221+k+offset)
    ax.plot(sim.scoutingcoordinates[:,0], sim.scoutingcoordinates[:,1],'k*')
    ax.plot(out.loc.xnm[notvld],out.loc.ynm[notvld],'y.')
    ax.plot(out.loc.xnm[vld],out.loc.ynm[vld],'m.')
    ax.plot(out.loc.xnm[vldcfr],out.loc.ynm[vldcfr],'bx')
    posfl = out.fluorophores.pos[-1,:,:].squeeze()
    ax.plot(posfl[:,0],posfl[:,1],'ro')
    ax.set_aspect('equal')
    ax.legend(['scouting','not vld', 'last itr vld','last itr vld +cfr', 'fluorophore'])
    ax.set_title(titles[k])
    ax.plot([-90, 10],[-70, -70],'k')
    ax.text(-50,-60,"100 µm")
    ax.set_axis_off()
    ax.set_xlim([-95, 100])
    ax.set_ylim([-85, 105])

In [ ]:
fig.tight_layout()

## Fluorohore density for dSTORM and filtering

In [ ]:
#make abberior simulator
fig = plt.figure()

photonbudget = 5000
reactivations = 2
brightnesses = 100
laserpower = 5

toff = np.array([0.5, 1, 2, 5, 10, 20, 50, 100, 200])*1e3  # ms

for k in range(len(toff)):
    sim.posgalvo = [0, 0, 0]
    sim.posEOD = [0, 0, 0]
    sim.time = 0
    fc = FlCollectionBlinking()
    # set parameterst for caged fluorophore, PAFP or similar
    switchpar = {}
    switchpar['brightness']=brightnesses*laserpower
    switchpar['toffsmlm'] = toff[k]
    switchpar['photonbudget'] = photonbudget
    switchpar['tonsmlm'] = 1e4 #  ms stays on, only bleached
    switchpar['activations'] = reactivations  # re activations
    switchpar['starton'] = -1 # fluorophores start in random on / off state, determined by tonsmlm, toffsmlm
    fc.setpar(**switchpar)

    # add fake NPCs
    fc.addstatic(makeNPC(pos=[0, 0, 0]))
    fc.reset()

    sim.fluorophores = fc

    out=sim.scoutingSequence(maxtime=maxtime)

    # plot results

    vld = (out.loc.vld==1) & (out.loc.itr==max(out.loc.itr))
    vldcfr = vld & (out.loc.cfr<cfrcutoff)
    notvld = (~vld) & (~vldcfr)
    # subplot(2,2,k+2);
    ax = fig.add_subplot(331+k)
    ax.plot(sim.scoutingcoordinates[:,0], sim.scoutingcoordinates[:,1],'k*')
    ax.plot(out.loc.xnm[notvld], out.loc.ynm[notvld], 'y.')
    ax.plot(out.loc.xnm[vld], out.loc.ynm[vld], 'm.')
    ax.plot(out.loc.xnm[vldcfr], out.loc.ynm[vldcfr], 'bx')
    posfl = out.fluorophores.pos[-1,:,:].squeeze()
    ax.plot(posfl[:,0],posfl[:,1],'ro')
    ax.set_aspect('equal')
    ax.legend(['scouting','not vld', 'last itr vld','last itr vld +cfr', 'fluorophore'])
    ax.set_title(f"toff (s): {toff[k]/1000}")
    ax.plot([-90, 10],[-70, -70],'k')
    ax.text(-50,-60,"100 µm")
    ax.set_axis_off()
    ax.set_xlim([-95, 100])
    ax.set_ylim([-85, 105])
fig.tight_layout()